In [14]:
import cv2 as cv
import numpy as np

# Transmitter

In [15]:
class Transmitter:
  def __init__(self, im, pn, mask_ratio):
    self.im = im
    self.pn = pn
    self.mask_ratio = mask_ratio
    self.image_size = im.shape[0]

  def Watermarking(self):

    f_im = np.fft.fft2(self.im)
    f_im_shifted = np.fft.fftshift(f_im)

    f_pn = np.fft.fft2(self.pn)
    f_pn_shifted = np.fft.fftshift(f_pn)

    # Create a low-pass filter mask
    rows, cols = self.im.shape
    crow, ccol = rows // 2, cols // 2
    radius = int(self.im.shape[0] * self.mask_ratio) # Adjust for desired frequency cutoff
    mask = np.zeros((rows, cols), np.uint8)
    cv.circle(mask, (ccol, crow), radius, 1, thickness=-1)

    # Apply low-pass filter on im (keep low frequencies)
    low_freqs_im = f_im_shifted * mask

    # Extract high frequencies from pn
    high_freqs_pn = f_pn_shifted * (1 - mask)


    # Combine low frequencies of im with high frequencies of pn
    watermarked_fft = low_freqs_im + high_freqs_pn

    # Inverse FFT to reconstruct the hybrid image
    watermarked_img = np.fft.ifft2(np.fft.ifftshift(watermarked_fft))
    watermarked_img = np.abs(watermarked_img)

    return watermarked_img


  def Watermarking_with_square_mask(self):

    f_im = np.fft.fft2(self.im)
    f_im_shifted = np.fft.fftshift(f_im)

    f_pn = np.fft.fft2(self.pn)
    f_pn_shifted = np.fft.fftshift(f_pn)

    # Create a low-pass filter mask
    mask = self.__Created_Square_Mask(self.im.shape[0] , self.mask_ratio)

    # Apply low-pass filter on im (keep low frequencies)
    low_freqs_im = f_im_shifted * mask

    # Extract high frequencies from pn
    high_freqs_pn = f_pn_shifted * (1 - mask)


    # Combine low frequencies of im with high frequencies of pn
    watermarked_fft = low_freqs_im + high_freqs_pn

    # Inverse FFT to reconstruct the hybrid image
    watermarked_img = np.fft.ifft2(np.fft.ifftshift(watermarked_fft))
    watermarked_img = np.abs(watermarked_img)

    return watermarked_img


  def Created_Masked_PN(self):
    fft_hybrid = np.fft.fft2(self.pn)
    fft_hybrid_shifted = np.fft.fftshift(fft_hybrid)

    mask = self.__Created_Mask(self.image_size, self.mask_ratio)

    high_freqs = fft_hybrid_shifted * (1 - mask)
    reconstructed_pn = np.fft.ifft2(np.fft.ifftshift(high_freqs))
    reconstructed_pn_abs = np.abs(reconstructed_pn)

    epsilon = 1e-8
    spectrum = np.log(reconstructed_pn_abs + epsilon)

    return spectrum

  def __Created_Mask(self, im_size , mask_ratio):
    rows, cols = im_size, im_size
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    mask_radius = int(mask_ratio * im_size)
    cv.circle(mask, (ccol, crow), mask_radius, 1, thickness=-1)
    return mask

  def __Created_Square_Mask(self, im_size, mask_ratio):
    rows, cols = im_size, im_size
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)

    mask_half_side = int((mask_ratio * im_size) / 2)

    top_left = (ccol - mask_half_side, crow - mask_half_side)
    bottom_right = (ccol + mask_half_side, crow + mask_half_side)

    cv.rectangle(mask, top_left, bottom_right, 1, thickness=-1)
    return mask

  def __normalized_image(self , im):
      # Find the minimum and maximum values in the matrix
      min_value = im.min()
      max_value = im.max()

      # Apply min-max normalization to each element
      normalized_im = (im - min_value) / (max_value - min_value)

      return normalized_im



# Receiver

In [16]:
class Receiver:
  def __init__(self, watermarked_img, mask_ratio):
    self.watermarked_img = watermarked_img
    self.mask_ratio = mask_ratio
    self.image_size = watermarked_img.shape[0]

  def PN_Reconstruction(self):
    fft_hybrid = np.fft.fft2(self.watermarked_img)
    fft_hybrid_shifted = np.fft.fftshift(fft_hybrid)

    #mask = self.__Created_Mask(image_size, mask_ratio)
    mask = self.__Created_Mask(self.image_size, self.mask_ratio)

    recovered_high_freqs = fft_hybrid_shifted * (1 - mask)
    reconstructed_pn = np.fft.ifft2(np.fft.ifftshift(recovered_high_freqs))
    reconstructed_pn_abs = np.abs(reconstructed_pn)

    epsilon = 1e-8
    spectrum = np.log(reconstructed_pn_abs + epsilon)

    return spectrum


  def PN_Reconstruction_with_square_mask(self):
    fft_hybrid = np.fft.fft2(self.watermarked_img)
    fft_hybrid_shifted = np.fft.fftshift(fft_hybrid)

    mask = self.__Created_Square_Mask(self.image_size, self.mask_ratio)

    recovered_high_freqs = fft_hybrid_shifted * (1 - mask)
    reconstructed_pn = np.fft.ifft2(np.fft.ifftshift(recovered_high_freqs))
    reconstructed_pn_abs = np.abs(reconstructed_pn)

    epsilon = 1e-8
    spectrum = np.log(reconstructed_pn_abs + epsilon)

    return spectrum


  def __Created_Mask(self, im_size , mask_ratio):
    rows, cols = im_size, im_size
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    mask_radius = int(mask_ratio * im_size)
    cv.circle(mask, (ccol, crow), mask_radius, 1, thickness=-1)
    return mask


  def __Created_Square_Mask(self, im_size, mask_ratio):
    rows, cols = im_size, im_size
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)

    mask_half_side = int((mask_ratio * im_size) / 2)

    top_left = (ccol - mask_half_side, crow - mask_half_side)
    bottom_right = (ccol + mask_half_side, crow + mask_half_side)

    cv.rectangle(mask, top_left, bottom_right, 1, thickness=-1)
    return mask

# Data

In [17]:
class Data:
  def __init__(self, num_samples, image_size, mask_ratio):
    self.num_samples = num_samples
    self.image_size = image_size
    self.mask_ratio = mask_ratio


  #Created data in boundary [0,1]
  def Created_Masked_PN_Arr(self, class_parameter):
    data = np.zeros((self.num_samples, self.image_size, self.image_size))

    for i in range(self.num_samples):
      data[i, :, :] = self.__Created_Masked_PN(class_parameter)

    return data

  def __Created_Masked_PN(self, frq):

    pn = self.created_wave_pattern(frq)

    fft_hybrid = np.fft.fft2(pn)
    fft_hybrid_shifted = np.fft.fftshift(fft_hybrid)

    mask = self.__Created_Mask(self.image_size, self.mask_ratio)

    high_freqs = fft_hybrid_shifted * (1 - mask)
    reconstructed_pn = np.fft.ifft2(np.fft.ifftshift(high_freqs))
    reconstructed_pn_abs = np.abs(reconstructed_pn)

    epsilon = 1e-8
    spectrum = np.log(reconstructed_pn_abs + epsilon)

    return spectrum

  def created_wave_pattern(self, freq):
    freq_variation = 1.5
    x = np.linspace(0, 2 * np.pi, self.image_size)
    y = np.linspace(0, 2 * np.pi, self.image_size)
    X, Y = np.meshgrid(x, y)

    # Apply small random frequency variations
    freq_x = freq + np.random.uniform(-freq_variation, freq_variation)
    freq_y = freq + np.random.uniform(-freq_variation, freq_variation)

    # Random phase shifts
    phase_shift_x = np.random.uniform(0, 2 * np.pi)
    phase_shift_y = np.random.uniform(0, 2 * np.pi)

    # Generate wave pattern with varied frequency and phase shift
    wave = np.sin(freq_x * X + phase_shift_x) * np.cos(freq_y * Y + phase_shift_y)

    return self.__normalized_image(wave)

  def __Created_Mask(self, im_size , mask_ratio):
    rows, cols = im_size, im_size
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    mask_radius = int(mask_ratio * im_size)
    cv.circle(mask, (ccol, crow), mask_radius, 1, thickness=-1)
    return mask

  def __Created_Square_Mask(self, im_size, mask_ratio):
    rows, cols = im_size, im_size
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)

    mask_half_side = int((mask_ratio * im_size) / 2)

    top_left = (ccol - mask_half_side, crow - mask_half_side)
    bottom_right = (ccol + mask_half_side, crow + mask_half_side)

    cv.rectangle(mask, top_left, bottom_right, 1, thickness=-1)
    return mask


  def __normalized_image(self , im):
      # Find the minimum and maximum values in the matrix
      min_value = im.min()
      max_value = im.max()

      # Apply min-max normalization to each element
      normalized_im = (im - min_value) / (max_value - min_value)

      return normalized_im


# Augmentation

In [18]:
from scipy.ndimage import gaussian_filter
from scipy.ndimage import shift
import cv2 as cv
import numpy as np


class Augmentation:
  def __init__(self, image):
    self.image = image


  def add_random_noise(self, noise_strength):
    #image = self._normalized_image(self.image)

    noise = np.random.normal(loc=0, scale=1, size=self.image.shape)  # Generate standard normal noise
    noise = (noise - np.min(noise)) / (np.max(noise) - np.min(noise))  # Normalize noise to [0,1]
    noise = 2 * noise_strength * (noise - 0.5)  # Scale to [-noise_strength, noise_strength]

    # Apply Gaussian blur to create correlated noise
    correlated_noise = gaussian_filter(noise, sigma=1)

    # Add noise to the image and clip to valid range
    image_noisy = self.image + correlated_noise
    image_noisy = np.clip(image_noisy, 0, 1)
    return image_noisy


  def scaling(self, factor):

    original_shape = self.image.shape

    # Downsample
    downsampled = cv.resize(self.image, (original_shape[1] // factor, original_shape[0] // factor), interpolation=cv.INTER_LINEAR)

    # Upsample
    upsampled = cv.resize(downsampled, (original_shape[1], original_shape[0]), interpolation=cv.INTER_LINEAR)

    return upsampled


  def shift_image(self, shift_x=0, shift_y=0):
    """
    Shift image by shift_x (cols) and shift_y (rows).
    Positive values move right/down, negative left/up.
    """
    return shift(self.image, shift=(shift_y, shift_x), mode='constant', cval=0)


  def circular_shift_attack(self, shift_value):
    return np.roll(self.image, shift_value, axis=1)  # axis=1 is for the x-direction (columns)


  def custom_shift_zero_padding(self, shift_value):

    im_shifted = np.zeros_like(self.image)

    # Perform the shift by slicing
    if shift_value > 0:
        im_shifted[:, shift_value:] = self.image[:, :-shift_value]
    elif shift_value < 0:
        im_shifted[:, :shift_value] = self.image[:, -shift_value:]

    return im_shifted


  def image_rotation(self, angle):
    """
    Simulates a rotation attack on a 2D grayscale image.

    Parameters:
    - image: 2D numpy array (grayscale)
    - angle: float, rotation angle in degrees (positive = counter-clockwise)

    Returns:
    - rotated_image: image rotated by the angle and resized back to original shape
    """
    height, width = self.image.shape
    center = (width // 2, height // 2)

    # Get rotation matrix
    rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)

    # Apply rotation
    rotated = cv.warpAffine(
        self.image,
        rotation_matrix,
        (width, height),
        flags=cv.INTER_LINEAR,
        borderMode=cv.BORDER_CONSTANT,
        borderValue=0  # fill black borders
    )

    return rotated



  def _normalized_image(self, im):
    # Find the minimum and maximum values in the matrix
    min_value = im.min()
    max_value = im.max()

    # Apply min-max normalization to each element
    normalized_im = (im - min_value) / (max_value - min_value)

    return normalized_im


# Transparency

In [19]:
class Transparency:
  def __init__(self, image, noisy_image):
    self.image = image
    self.noisy_image = noisy_image

  def MSE(self):
    # Ensure both images have the same shape
    #assert self.image.shape == self.noisy_image.shape, "Images must have the same dimensions"

    # Compute MSE
    error = np.mean((self.image - self.noisy_image) ** 2)
    return error

  def PSNR(self , max_pixel=1.0):
    mse_value = np.mean((self.image - self.noisy_image) ** 2)

    if mse_value == 0:
        return float('inf')  # If images are identical, PSNR is infinite

    psnr_value = 10 * np.log10((max_pixel ** 2) / mse_value)
    return psnr_value

  def SNR(self):
    signal_power = np.mean(self.image ** 2)  # Power of the signal (original image)
    noise_power = np.mean((self.image - self.noisy_image) ** 2)  # Power of the noise

    if noise_power == 0:
        return float('inf')  # If there's no noise, SNR is infinite

    snr_value = 10 * np.log10(signal_power / noise_power)
    return snr_value